In [1]:
# Score calculator for Santa 2025
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union

getcontext().prec = 30

# Tree geometry
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        
        trunk_w, trunk_h = Decimal('0.15'), Decimal('0.2')
        base_w, mid_w, top_w = Decimal('0.7'), Decimal('0.4'), Decimal('0.25')
        tip_y, tier_1_y, tier_2_y, base_y = Decimal('0.8'), Decimal('0.5'), Decimal('0.25'), Decimal('0.0')
        trunk_bottom_y = -trunk_h
        
        initial_polygon = Polygon([
            (0, float(tip_y)),
            (float(top_w/2), float(tier_1_y)), (float(top_w/4), float(tier_1_y)),
            (float(mid_w/2), float(tier_2_y)), (float(mid_w/4), float(tier_2_y)),
            (float(base_w/2), float(base_y)),
            (float(trunk_w/2), float(base_y)), (float(trunk_w/2), float(trunk_bottom_y)),
            (float(-trunk_w/2), float(trunk_bottom_y)), (float(-trunk_w/2), float(base_y)),
            (float(-base_w/2), float(base_y)),
            (float(-mid_w/4), float(tier_2_y)), (float(-mid_w/2), float(tier_2_y)),
            (float(-top_w/4), float(tier_1_y)), (float(-top_w/2), float(tier_1_y)),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x), yoff=float(self.center_y))

def calculate_score(csv_path):
    df = pd.read_csv(csv_path)
    df['x'] = df['x'].astype(str).str.strip().str.lstrip('s').astype(float)
    df['y'] = df['y'].astype(str).str.strip().str.lstrip('s').astype(float)
    df['deg'] = df['deg'].astype(str).str.strip().str.lstrip('s').astype(float)
    df[['group_id', 'item_id']] = df['id'].str.split('_', n=2, expand=True)
    df['group_id'] = df['group_id'].astype(int)
    
    total_score = 0
    per_n_scores = {}
    
    for n, group in df.groupby('group_id'):
        trees = [ChristmasTree(row['x'], row['y'], row['deg']) for _, row in group.iterrows()]
        all_polygons = [t.polygon for t in trees]
        bounds = unary_union(all_polygons).bounds
        side = max(bounds[2] - bounds[0], bounds[3] - bounds[1])
        score_n = (side ** 2) / n
        per_n_scores[n] = {'side': side, 'score': score_n}
        total_score += score_n
    
    return total_score, per_n_scores

# Test with sample submission
score, per_n = calculate_score('/home/data/sample_submission.csv')
print(f'Sample submission score: {score:.6f}')
print(f'\nWorst 10 N values by score contribution:')
sorted_scores = sorted(per_n.items(), key=lambda x: x[1]['score'], reverse=True)[:10]
for n, data in sorted_scores:
    print(f'  N={n}: side={data["side"]:.4f}, score={data["score"]:.4f}')

Sample submission score: 173.652299

Worst 10 N values by score contribution:
  N=8: side=3.4411, score=1.4802
  N=13: side=4.1474, score=1.3231
  N=9: side=3.4411, score=1.3157
  N=14: side=4.1474, score=1.2286
  N=7: side=2.9016, score=1.2028
  N=10: side=3.4411, score=1.1841
  N=15: side=4.1474, score=1.1467
  N=11: side=3.4411, score=1.0765
  N=16: side=4.1474, score=1.0751
  N=4: side=2.0393, score=1.0396


In [2]:
# Check a snapshot submission
import os
snapshot_dir = '/home/nonroot/snapshots/santa-2025/21331543270/submission/'
if os.path.exists(snapshot_dir + 'submission.csv'):
    score, per_n = calculate_score(snapshot_dir + 'submission.csv')
    print(f'Snapshot submission score: {score:.6f}')
    print(f'\\nWorst 10 N values by score contribution:')
    sorted_scores = sorted(per_n.items(), key=lambda x: x[1]['score'], reverse=True)[:10]
    for n, data in sorted_scores:
        print(f'  N={n}: side={data["side"]:.4f}, score={data["score"]:.4f}')
else:
    print("Submission not found")

Submission not found


In [3]:
# Find all submission.csv files in snapshots
import glob
submissions = glob.glob('/home/nonroot/snapshots/santa-2025/*/submission/submission.csv')
print(f"Found {len(submissions)} submissions")

# Check first few
for sub in submissions[:3]:
    try:
        score, _ = calculate_score(sub)
        print(f'{sub}: {score:.6f}')
    except Exception as e:
        print(f'{sub}: Error - {e}')

Found 82 submissions


/home/nonroot/snapshots/santa-2025/21116303805/submission/submission.csv: 70.676102


/home/nonroot/snapshots/santa-2025/21328309254/submission/submission.csv: 70.647327


/home/nonroot/snapshots/santa-2025/21121776553/submission/submission.csv: 70.936674


In [4]:
# Find best submission across all snapshots
best_score = float('inf')
best_path = None
all_scores = []

for sub in submissions:
    try:
        score, _ = calculate_score(sub)
        all_scores.append((score, sub))
        if score < best_score:
            best_score = score
            best_path = sub
    except Exception as e:
        pass

print(f"Best score: {best_score:.6f}")
print(f"Best path: {best_path}")

# Show top 5
all_scores.sort()
print("\\nTop 5 submissions:")
for score, path in all_scores[:5]:
    print(f"  {score:.6f}: {path.split('/')[-3]}")

Best score: 70.572798
Best path: /home/nonroot/snapshots/santa-2025/21145966992/submission/submission.csv
\nTop 5 submissions:
  70.572798: 21145966992
  70.615745: 21329067673
  70.615745: 21328310479
  70.616145: 21322576827
  70.619825: 21322576451
